In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



# Let's start by readidng the data from the csv files

In [ ]:
train=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv',index_col=False)
test=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv',index_col=False)
sample=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv',index_col=False)

Let's see a few radom samples from the train dataframe

In [ ]:
train.sample(5)

lets check if we have any missing values in our dataframes

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
y=train.target.values

In [ ]:
y[:5]

let's import the packages we need to build our model

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import GRU,Bidirectional,Embedding,Dense,Flatten,GlobalMaxPool1D,LSTM
from keras.models import Sequential


In [ ]:
num_words=3000
max_len=200
tokenizer=Tokenizer(num_words,oov_token='<oov>')
tokenizer.fit_on_texts(train.question_text)
train_sequences=tokenizer.texts_to_sequences(train.question_text)
test_sequences=tokenizer.texts_to_sequences(test.question_text)

# let's pad the sequences to make them of equal length

train_padded=pad_sequences(train_sequences,maxlen=max_len)
test_padded=pad_sequences(test_sequences,maxlen=max_len)

In [ ]:
model=Sequential([Embedding(num_words,128,input_length=max_len),
                 Bidirectional(LSTM(128,return_sequences=True)),
                               GlobalMaxPool1D(),
                              Dense(128,activation='relu'),
                            
                               
                              Dense(1,activation='sigmoid')])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics='accuracy')

In [ ]:
batch_size=400
epochs=3
model.fit(train_padded,y,epochs=epochs,batch_size=batch_size,steps_per_epoch=len(train)//batch_size,validation_split=.1)

In [ ]:
predicted=model.predict(test_padded)

In [ ]:
test['prediction']=predicted

In [ ]:
test.sample(5)

In [ ]:
test.drop(['question_text'],axis=1,inplace=True)

In [ ]:
test.sample(5)

In [ ]:
test['prediction']=test['prediction'].apply(lambda x: 0 if x<.5 else 1)

In [ ]:
test.prediction.value_counts()

In [ ]:
test.to_csv('submission.csv',index=False)